# TVB-NEST: Bridging multiscale activity by co-simulation

## Step-by-step learn how to perform a co-simulation embedding spiking neural networks into large-scale brain networks using TVB.

In [ ]:
from IPython.display import Image, display
display(Image(filename='pics/ConceptGraph1.png',  width=1000, unconfined=False))

In [ ]:
display(Image(filename='pics/ConceptGraph2.png',  width=1000, unconfined=False))

## tvb-multiscale toolbox:

### https://github.com/the-virtual-brain/tvb-multiscale

For questions use the git issue tracker, or write an e-mail to me: dionysios.perdikis@charite.de

# TVB - NEST co-simulation 

## Jansen - Rit TVB mean field model

For every region node $n\prime$ modelled as a mean-field node in TVB:

Population activity dynamics (1 pyramidal excitatory (0,3) and 1 excitatory (1,4) and 1 inhibitory (2,5) interneurons populations):


\begin{align}
\dot{y_0} &= y_3 \\
\dot{y_1} &= y_4 \\
\dot{y_2} &= y_5 \\
\dot{y_3} &= A a\,S[y_1 - y_2] - 2a\,y_3 - a^2\, y_0 \\
\dot{y_4} &= A a \,[p(t) + \alpha_2 J + S[\alpha_1 J\,y_0]+ c_0] -2a\,y - a^2\,y_1 \\
\dot{y_5} &= B b (\alpha_4 J\, S[\alpha_3 J \,y_0]) - 2 b\, y_5 - b^2\,y_2 \\
     S[v] &= \frac{2\nu_{max}}{1 + \exp^{r(v_0 - v)}}
\end{align}

## Spiking network model in NEST

using "iaf_cond_alpha" spiking neuronal model.

## TVB to NEST coupling

TVB couples to NEST via instantaneous spike rate $ scale * rate(t) + translation$, 

Inhomogeneous spike generator NEST devices are used as TVB "proxy" nodes and generate independent Poisson-random spike trains 

$ \left[ \sum_k \delta(t-\tau_{n\prime n}-{t_j}^k) \right]_{j \in n\prime} $

Alternatively, the spike trains are generated outside NEST using the Elephant software and inserted to NEST via spike generator devices.

The rate is can be computed in TVB using the default SigmoidalJansenRit coupling model, which outputs coupling using the $y_1-y_2$ state variables from each region node:

$ rate_n(t) = a*\sum_{n\prime}\frac{2c_{max}}{1 + \exp^{ r (midpoint - (y_{1_{n\prime}} - y_{2_{n\prime}} ) ) } }$, 

in which case one independent Poisson-random spike train is generated for each receiver NEST node.

Alternatively, the $y_1, y_2$ state variables are sent from TVB for every node $n\prime$ and the sigmoidal function of $y_1 - y_2$ is computed at a dedicated transformer class: 

$ rate_{n\prime}(t) = \frac{2\, c_{max}}{1 + \exp^{r(v0 - (y1_{n\prime}-y2_{n\prime}))}} $

in order to generate $n'$ independent Poisson-random spike trains for each sender TVB node, i.e., in the latter case large scale coupling is computed via summation in NEST.


## NEST to TVB update

For a SPIKES model, a NEST spike recorder device is used to count spike for each TVB time step, and convert it to an instantaneous population mean rate

$ rate_{_{n}}(t) =  \frac{\sum_j\left[ \sum_k \delta(t-\tau_n-{t_j}^k) \right]_{j \in P_n}}{N * dt} $ 

where P_n is the population P at NEST node $n$ comprising $N$ neurons and rate is computed in  spikes/sec.

After passing via a linear transform $ scale_p * rate_n(t) + translation_p $, it enters a transformer of an inverse Jansen & Rit sigmoidal function: 
$y_{p_n}(t) = v_0 - \log(\frac{2*R_{max}}{\min(\max(rate_n(t), R_{min}), R_{max})} - 1))/r $

This update process concerns only the TVB region nodes that are simulated exclusively in NEST, as spiking networks. All the rest of TVB nodes will follow the equations of the mean field model described above.


Instead in the case of a POTENTIAL model, a NEST multimeter device records membrane potentials at each TVB time step, and we compute the mean field membrane potential of each population:

$ V_{p_n}(t) =  \frac{\sum_j\left[ V_j \right]_{j \in P_n}}{N} $ 

where P_n is the population P at NEST node $n$ comprising $N$ neurons.

Finally, the TVB mean field state variable activity is computed via a linear transform as:

$ y_{p_n}(t) =scale_p * V_{p_n}(t) + translation_p $


## Simulator loop

### Simulating several (i.e., minimally 2) NEST time steps for every 1 TVB time step for stable integration

### Synchronizaion every minimum delay time between the two simulators.

# WORKFLOW:

In [ ]:
import os
from collections import OrderedDict
import time
import numpy as np

from tvb.basic.profile import TvbProfile
TvbProfile.set_profile(TvbProfile.LIBRARY_PROFILE)

from tvb_multiscale.tvb_nest.config import *

# ----------- Simulation options ----------------
SIM_MODE = "tvb-nest"  # "tvb-nest"  for multiscale cosimulation, "tvb" ("nest") for only TVB (NEST) simulation, respectively
NEST_MODEL_BUILDERS = "opinionated" #  , "opinionated", "nonopinionated", None
TVB_NEST_COUPLING_MODE = "TVB"  # "TVB", "NEST"
NEST_TVB_MODEL = "SPIKES" # "SPIKES", "POTENTIAL"
# -----------------------------------------------

work_path = os.getcwd()
outputs_path = os.path.join(work_path, "outputs/JansenRit")
if NEST_MODEL_BUILDERS is None:
    outputs_path += "NoNestBuilders"
elif NEST_MODEL_BUILDERS == "opinionated":
    outputs_path += "OpinionBuilders"
elif NEST_MODEL_BUILDERS == "nonopinionated":
    outputs_path += "NonOpinionBuilders"
    
if SIM_MODE.lower() == "nest":
    outputs_path += "NESTonly"
elif SIM_MODE.lower() == "tvb":
    outputs_path += "TVBonly"

outputs_path += "_COUPLING_MODE_%s" % TVB_NEST_COUPLING_MODE
outputs_path += "_NEST_TVB_MODEL_%s" % NEST_TVB_MODEL

config = Config(output_base=outputs_path)
config.figures.SHOW_FLAG = True 
config.figures.SAVE_FLAG = True
config.figures.FIG_FORMAT = 'png'
config.figures.DEFAULT_SIZE= config.figures.NOTEBOOK_SIZE
FIGSIZE = config.figures.DEFAULT_SIZE

from tvb_multiscale.core.plot.plotter import Plotter
plotter = Plotter(config.figures)

# For interactive plotting:
# %matplotlib notebook  

# Otherwise:
%matplotlib inline 

## 1. Load structural data <br> (minimally a TVB connectivity)  <br> & prepare TVB simulator  <br> (region mean field model, integrator, monitors etc)

In [ ]:
# This would run on TVB only before creating any multiscale cosimulation interface connections.

from tvb_multiscale.core.tvb.cosimulator.models.jansen_rit import JansenRit

# Create a TVB simulator and set all desired inputs
# (connectivity, model, surface, stimuli etc)
# We choose all defaults in this example
# -----------------------------------Wilson Cowan oscillatory regime--------------------------------
model_params = {
    }


# -----------------------------------Build cosimunlator manually--------------------------------
from tvb_multiscale.core.tvb.cosimulator.cosimulator_serial import CoSimulatorSerial
from tvb.datatypes.connectivity import Connectivity
from tvb_multiscale.core.tvb.cosimulator.models.jansen_rit import SigmoidalJansenRit
from tvb.simulator.integrators import HeunStochastic
from tvb.simulator.monitors import Raw, Bold, EEG

simulator = CoSimulatorSerial()

simulator.model = JansenRit(**model_params)
simulator.model.variables_of_interest = np.array(simulator.model.state_variables)[np.arange(simulator.model.nvar)].tolist()
simulator.integrator = HeunStochastic()
simulator.integrator.dt = 0.1  # 1.0 for faster simulation, but check for the integration stability!
simulator.integrator.noise.nsig = np.array([config.DEFAULT_NSIG] * simulator.model.nvar) # 0.001


# Load connectivity
# config.DEFAULT_CONNECTIVITY_ZIP = "/home/docker/packages/tvb_data/tvb_data/mouse/allen_2mm/ConnectivityAllen2mm.zip"                                  
connectivity = Connectivity.from_file(config.DEFAULT_CONNECTIVITY_ZIP)

# Normalize connectivity weights
connectivity.weights = connectivity.scaled_weights(mode="region")
connectivity.weights /= np.percentile(connectivity.weights, 99)
# connectivity.weights[connectivity.weights > 1.0] = 1.0

# connectivity.tract_lengths = np.maximum(connectivity.speed * simulator.integrator.dt, 
#                                         connectivity.tract_lengths)

connectivity.configure()

simulator.connectivity = connectivity

simulator.coupling = SigmoidalJansenRit()

simulator.initial_conditions = np.random.normal(size=(1, simulator.model.nvar, connectivity.number_of_regions, 1))

mon_raw = Raw(period=1.0, )  # ms
simulator.monitors = (mon_raw, )

simulator.configure()



# # -----------------------------------Or use the CoSimulator builder--------------------------------
# from tvb_multiscale.core.tvb.cosimulator.cosimulator_builder import CoSimulatorSerialBuilder

# simulator_builder = CoSimulatorSerialBuilder()
# simulator_builder.config = config
# simulator_builder.model = WilsonCowan()
# simulator_builder.model_params = model_params
# simulator_builder.initial_conditions = np.zeros((1, 1, 1, 1))

# simulator_builder.configure()
# simulator_builder.print_summary_info_details(recursive=1)

# simulator = simulator_builder.build()



# simulator.print_summary_info_details(recursive=1)

# # Plot TVB connectome:
# plotter.plot_tvb_connectivity(simulator.connectivity);


# Serializing TVB cosimulator is necessary for parallel cosimulation:
from tvb_multiscale.core.utils.file_utils import dump_pickled_dict
from tvb_multiscale.core.tvb.cosimulator.cosimulator_serialization import serialize_tvb_cosimulator
sim_serial_filepath = os.path.join(config.out.FOLDER_RES, "tvb_serial_cosimulator.pkl")
sim_serial = serialize_tvb_cosimulator(simulator)
display(sim_serial)

# Dumping the serialized TVB cosimulator to a file will be necessary for parallel cosimulation.
dump_pickled_dict(sim_serial, sim_serial_filepath)


## 2. Build and connect the NEST network model <br> (networks of spiking neural populations for fine-scale <br>regions, stimulation devices, spike detectors etc)

In [ ]:
# This would run on NEST only before creating any multiscale cosimulation interface connections.
# Here it is assumed that the TVB simulator is already created and we can get some of its attributes, 
# either by directly accessing it, or via serialization.

nest = None
nest_network = None
nest_nodes_inds = np.array([])
    
if "nest" in SIM_MODE.lower():

    # Build a NEST network model with the corresponding builder
    from tvb_multiscale.tvb_nest.nest_models.builders.nest_factory import load_nest, configure_nest_kernel

    from tvb_multiscale.core.utils.file_utils import load_pickled_dict
    sim_serial = load_pickled_dict(sim_serial_filepath)

    # Load NEST and use defaults to configure its kernel:
    nest = configure_nest_kernel(load_nest(config=config), config)

    # Select the brain region nodes to place spiking networks at:
    nest_nodes_inds = np.array([0, 1]) 

    jansen_rit_sigmoidal_coupling_fun = lambda source_node_ind, target_node_ind: \
                                            sim_serial['coupling.cmax'][0] * sim_serial['coupling.a'][0] *\
                                            sim_serial['connectivity.weights'][target_node_ind, source_node_ind]
            
    delay_fun =  lambda source_node_ind, target_node_ind: \
                    np.maximum(np.maximum(sim_serial['integrator.dt'], config.DEFAULT_SPIKING_MIN_DELAY),
                               sim_serial['connectivity.delays'][source_node_ind, target_node_ind]) 
            
    if NEST_MODEL_BUILDERS:

        if NEST_MODEL_BUILDERS == "opinionated":

            # ------------------- Instantiating an opinionated nest network builder for this model, ------------------- 
            # using all default parameters for this example

            from tvb_multiscale.tvb_nest.nest_models.models.jansen_rit import JansenRitBuilder

            nest_model_builder = JansenRitBuilder(sim_serial,  # simulator, 
                                                  spiking_nodes_inds=nest_nodes_inds, 
                                                  spiking_simulator=nest, config=config)

            # ... or modifying some of the builder's attributes:
            nest_model_builder.w_pe = 10*sim_serial['model.a_1'][0]  # 10*simulator.model.a_1[0]
            nest_model_builder.w_pi = 10*sim_serial['model.a_3'][0]  # 10*simulator.model.a_3[0]
            nest_model_builder.w_ep = 1.0 
            nest_model_builder.w_ip = -1.0 
            nest_model_builder.output_devices_record_to = "memory"  # "ascii"
            nest_model_builder.population_order = 100
            nest_model_builder.tvb_to_spiking_dt_ratio = 2 # 2 NEST integration steps for 1 TVB integration step
            nest_model_builder.def_min_delay = config.DEFAULT_SPIKING_MIN_DELAY
            nest_model_builder.monitor_period = 1.0

        else:

            # ------ Alternatively, instantiating a non-opinionated nest network builder for this model, ----------------- 
            # ... and setting desired network description:

            from tvb_multiscale.tvb_nest.nest_models.builders.base import NESTNetworkBuilder

            nest_model_builder = NESTNetworkBuilder(sim_serial, # simulator, 
                                                    spiking_nodes_inds=nest_nodes_inds, 
                                                    spiking_simulator=nest, config=config)
            nest_model_builder.output_devices_record_to = "memory"  # "ascii"
            nest_model_builder.population_order = 100
            nest_model_builder.tvb_to_spiking_dt_ratio = 2 # 2 NEST integration steps for 1 TVB integration step
            nest_model_builder.def_min_delay = config.DEFAULT_SPIKING_MIN_DELAY
            nest_model_builder.monitor_period = 1.0

            # Set populations:
            nest_model_builder.populations = []
            for pop in ["E", "Ein", "Iin"]:
                nest_model_builder.populations.append(
                    {"label": pop, 
                     "model": config.DEFAULT_SPIKING_MODEL,  # "iaf_cond_alpha" by default
                     # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                     "params": {},   # parameters for NEST neuronal model
                     "scale": 1.0,   # nest_model_builder.multiply population_order for the exact populations' size
                     # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                     "nodes": None}) # None means "all" -> building this population to all spiking_nodes_inds


            # "static_synapse" by default:
            synapse_model = config.DEFAULT_CONNECTION["synapse_model"] 
            # Default conn_spec: {'rule': "all_to_all", "allow_autapses": True, 'allow_multapses': True}
            conn_spec =  config.DEFAULT_CONNECTION["conn_spec"]  

            # Set populations' connections within brain region nodes
            nest_model_builder.populations_connections = [
                {"source": "E", "target": "Ein",  # E -> Ein
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                 "synapse_model": synapse_model,
                 "conn_spec": conn_spec, 
                 "weight": 10*sim_serial['model.a_1'][0], # simulator.model.a_1[0],
                 "delay": nest_model_builder.def_min_delay,
                 "receptor_type": 0, 
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                 "nodes": None}, # None means "all" -> performing this connection to all spiking_nodes_inds
                {"source": "E", "target": "Iin",  # E -> Iin
                 "synapse_model": synapse_model,
                 "conn_spec": conn_spec, 
                 "weight": 10*sim_serial['model.a_3'][0], # simulator.model.a_3[0],
                 "delay": nest_model_builder.def_min_delay,
                 "receptor_type": 0, 
                 "nodes": None},
                {"source": "Iin", "target": "E",  # Iin -> E
                 "synapse_model": synapse_model,
                 "conn_spec": conn_spec, 
                 "weight": -1.0, # 
                 "delay": nest_model_builder.def_min_delay,
                 "receptor_type": 0, 
                 "nodes": None},
                {"source": "Ein", "target": "E",  # Ein -> E
                 "synapse_model": synapse_model,
                 "conn_spec": conn_spec, 
                 "weight": 1.0, # 
                 "delay": nest_model_builder.def_min_delay,
                 "receptor_type": 0, 
                 "nodes": None}
            ]
            
            # Set populations' connections among brain region node:
            nest_model_builder.nodes_connections = [
                {"source": "Ein", "target": "E",
                 #--------- Possibly functions of (source_node_ind, target_node_ind, *args, **kwargs) -------------
                 "synapse_model": synapse_model,
                 "conn_spec": conn_spec,
                 # ...using TVB connectome weights:
                 "weight": jansen_rit_sigmoidal_coupling_fun,
                 # ...using TVB connectome delays
                 "delay": delay_fun,   
                 "receptor_type": 0, 
                 #--------- Possibly functions of (source_node_ind, target_node_ind, *args, **kwargs) -------------
                 "source_nodes": None,  # None means "all" -> performing this connection from all spiking_nodes_inds
                 "target_nodes": None},  # None means "all" -> performing this connection to all spiking_nodes_inds
                 {"source": "Iin", "target": "E",
                 #--------- Possibly functions of (source_node_ind, target_node_ind, *args, **kwargs) -------------
                 "synapse_model": synapse_model,
                 "conn_spec": conn_spec,
                 # ...using TVB connectome weights:
                 "weight": lambda source_node_ind, target_node_ind: \
                              -jansen_rit_sigmoidal_coupling_fun(source_node_ind, target_node_ind),
                 # ...using TVB connectome delays
                 "delay": delay_fun,   
                 "receptor_type": 0, 
                 #--------- Possibly functions of (source_node_ind, target_node_ind, *args, **kwargs) -------------
                 "source_nodes": None,  # None means "all" -> performing this connection from all spiking_nodes_inds
                 "target_nodes": None}  # None means "all" -> performing this connection to all spiking_nodes_inds
            ]

            # Set output recorder devices:
            params_spike_recorder = config.NEST_OUTPUT_DEVICES_PARAMS_DEF["spike_recorder"].copy()
            params_spike_recorder["record_to"] = nest_model_builder.output_devices_record_to
            params_multimeter = config.NEST_OUTPUT_DEVICES_PARAMS_DEF["multimeter"].copy()
            params_multimeter["record_to"] = nest_model_builder.output_devices_record_to
            params_multimeter["interval"] = nest_model_builder.monitor_period
            nest_model_builder.output_devices = [
                {"model": "spike_recorder", 
                 "connections": {"E": "E",      # Record spikes with label "E" from populations "E"
                                 "Ein": "Ein",  # Record spikes with label "Ein" from populations "Ein"
                                 "Iin": "Iin"}, # Record spikes with label "Iin" from populations "Iin"
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                 "params": params_spike_recorder,
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                 "nodes": None},  # None means all here -> recording from all spiking_nodes_inds
                {"model": "multimeter", 
                 "connections": {"E_ts": "E",       # Record time series with label "E_ts" from populations "E"
                                 "Ein_ts": "Ein",  # Record time series with label "Ein_ts" from populations "Ein"
                                 "Iin_ts": "Iin"}, # Record time series with label "Iin_ts" from populations "Iin"
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                 "params": params_multimeter,
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                 "nodes": None},  # None means all here -> recording from all spiking_nodes_inds

            ]


            # Set input stimulation devices:
            nest_model_builder.input_devices = [
                {"model": "poisson_generator",
                "connections": {"Stimulus": "E"}, # connect stimulus "Stimulus" to populations "E"
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                 "params": {"rate": 7500.0, "origin": 0.0, "start": nest_model_builder.spiking_dt}, 
                 "weights": 1.0,
                 "delays": nest_model_builder.def_min_delay,
                 "receptor_type": 0,
                 # ---------------- Possibly functions of spiking_nodes_inds --------------------------
                "nodes": None  # None means all here -> stimulating all spiking_nodes_inds
                }

            ]


        nest_model_builder.configure()

        nest_network = nest_model_builder.build()



    else:

        # ------------------- Construct the NEST network model manually ------------------- 

        from tvb_multiscale.tvb_nest.nest_models.network import NESTNetwork
        from tvb_multiscale.tvb_nest.nest_models.brain import NESTBrain
        from tvb_multiscale.tvb_nest.nest_models.region_node import NESTRegionNode
        from tvb_multiscale.tvb_nest.nest_models.population import NESTPopulation
        from tvb_multiscale.core.spiking_models.devices import DeviceSet
        from tvb_multiscale.tvb_nest.nest_models.devices import NESTSpikeRecorder, NESTMultimeter
        from tvb_multiscale.tvb_nest.nest_models.devices import NESTPoissonGenerator


        # First configure NEST kernel:
        nest.SetKernelStatus({"resolution": 0.05})

        print("Building NESTNetwork...")

        # Create NEST network...
        nest_network = NESTNetwork(nest)


        # ...starting from neuronal populations located at specific brain regions...
        nest_network.brain_regions = NESTBrain()
        for node_ind in nest_nodes_inds:
            region_name = sim_serial['connectivity.region_labels'][node_ind]
            # region_name = simulator.connectivity.region_labels[node_ind]
            if region_name not in nest_network.brain_regions.keys():
                nest_network.brain_regions[region_name] = NESTRegionNode(label=region_name)
            for pop in ["E", "Ein", "Iin"]:
                nest_network.brain_regions[region_name][pop] = \
                           NESTPopulation(nest.Create(config.DEFAULT_SPIKING_MODEL, 100), # possible NEST model params as well here
                                          nest, label=pop, brain_region=region_name)
                print("\n...created: %s..." % nest_network.brain_regions[region_name][pop].print_str())

        # "static_synapse" by default:
        synapse_model = config.DEFAULT_CONNECTION["synapse_model"] 
        # Default 
        conn_spec = {'rule': "all_to_all", "allow_autapses": True, 'allow_multapses': True}


        # Connecting populations...
        for src_node_ind in nest_nodes_inds: 
            src_node_lbl = sim_serial['connectivity.region_labels'][src_node_ind]
            # src_node_lbl = simulator.connectivity.region_labels[src_node_ind]
            for trg_node_ind in nest_nodes_inds: 
                trg_node_lbl = sim_serial['connectivity.region_labels'][trg_node_ind]
                # trg_node_lbl = simulator.connectivity.region_labels[trg_node_ind]
                if src_node_ind == trg_node_ind:
                    # ...within brain regions...:
                    for src_pop, trg_pop, w in zip(["E",   "E",   "Ein", "Iin"], 
                                                   ["Ein", "Iin", "E",   "E"], 
                                                   [sim_serial['model.a_1'][0].item(),  # simulator.model.c_ee[0].item(), 
                                                    sim_serial['model.a_3'][0].item(),  # simulator.model.c_ei[0].item(), 
                                                    1.0,  
                                                    -1.0  
                                                   ]):
                        nest.Connect(nest_network.brain_regions[src_node_lbl][src_pop].nodes, 
                                     nest_network.brain_regions[src_node_lbl][trg_pop].nodes, 
                                     syn_spec={"synapse_model": synapse_model, 
                                               "weight": 10*w, "delay": config.DEFAULT_SPIKING_MIN_DELAY, "receptor_type": 0}, 
                                     conn_spec=conn_spec)
                        print("\n...connected populations %s -> %s in brain region %s..." 
                              % (src_pop, trg_pop, src_node_lbl))
                else:
                    for src_pop, trg_pop, w in zip(["Ein", "Iin"], 
                                                   ["E",   "E"], 
                                                   [1.0, -1.0]):
                        # ...between brain regions...:
                        nest.Connect(nest_network.brain_regions[src_node_lbl][src_pop].nodes, 
                                     nest_network.brain_regions[src_node_lbl][trg_pop].nodes, 
                                     syn_spec={"synapse_model": synapse_model, 
                                               "weight": w*jansen_rit_sigmoidal_coupling_fun(src_node_ind, trg_node_ind), 
                                               "delay": delay_fun(src_node_ind, trg_node_ind),
                                               "receptor_type": 0}, 
                                     conn_spec=conn_spec)
                        print("\n...connected populations %s - %s -> %s - %s..." % 
                              (src_pop, src_node_lbl, trg_pop, trg_node_lbl))


        # Create output recorder devices:
        params_spike_recorder = config.NEST_OUTPUT_DEVICES_PARAMS_DEF["spike_recorder"].copy()
        params_spike_recorder["record_to"] = "memory"
        params_multimeter = config.NEST_OUTPUT_DEVICES_PARAMS_DEF["multimeter"].copy()
        params_multimeter["record_to"] = "memory"
        params_multimeter["interval"] = 1.0
        for pop, pop_ts in zip(["E", "Ein", "Iin"], ["E_ts", "Ein_ts", "Iin_ts"]):
            nest_network.output_devices[pop] = DeviceSet(label=pop, model="spike_recorder")
            nest_network.output_devices[pop_ts] = DeviceSet(label=pop_ts, model="multimeter")
            for node_ind in nest_nodes_inds:
                region_name = sim_serial['connectivity.region_labels'][node_ind]
                # region_name = simulator.connectivity.region_labels[node_ind]

                # Create and connect population spike recorder for this region:
                nest_network.output_devices[pop][region_name] = \
                    NESTSpikeRecorder(nest.Create("spike_recorder", 1, params=params_spike_recorder), 
                                      nest, model="spike_recorder", label=pop, brain_region=region_name)
                nest.Connect(nest_network.brain_regions[region_name][pop].nodes, 
                             nest_network.output_devices[pop][region_name].device)
                nest_network.output_devices[pop].update() # update DeviceSet after the new NESTDevice entry
                print("\n...created spike_recorder device for population %s in brain region %s..." % 
                      (pop, region_name))

                # Create and connect population multimeter for this region:
                nest_network.output_devices[pop_ts][region_name] = \
                    NESTMultimeter(nest.Create("multimeter", 1, params=params_multimeter), 
                                   nest, model="multimeter", label=pop_ts, brain_region=region_name)
                nest.Connect(nest_network.output_devices[pop_ts][region_name].device, 
                             nest_network.brain_regions[region_name][pop].nodes)
                nest_network.output_devices[pop_ts].update() # update DeviceSet after the new NESTDevice entry
                print("\n...created multimeter device %s for population %s in brain region %s..." % 
                      (pop_ts, pop, region_name))


        # Create input stimulation devices:
        nest_network.input_devices["Stimulus"] = DeviceSet(label="Stimulus", model="poisson_generator")
        nest_dt = nest.GetKernelStatus('resolution')
        for node_ind in nest_nodes_inds:
                region_name = sim_serial['connectivity.region_labels'][node_ind]
                # region_name = simulator.connectivity.region_labels[node_ind]
                # Create and connect population spike recorder for this region:
                nest_network.input_devices["Stimulus"][region_name] = \
                    NESTPoissonGenerator(nest.Create("poisson_generator", 1, 
                                                     params={"rate": 7500.0, "origin": 0.0, "start": nest_dt}), 
                                         nest, model="poisson_generator", label="Stimulus", brain_region=region_name)
                nest.Connect(nest_network.input_devices["Stimulus"][region_name].device,
                             nest_network.brain_regions[region_name]["E"].nodes, 
                             syn_spec={"weight": 1.0, "delay": config.DEFAULT_SPIKING_MIN_DELAY})
                nest_network.input_devices["Stimulus"].update()  # update DeviceSet after the new NESTDevice entry
                print("\n...created poisson_generator device for population E in brain region %s..." % region_name)   



    # Configure NESTNetwork class:
    nest_network.configure()
    nest_network.print_summary_info_details(recursive=3, connectivity=True)    
    


## 3. Build the TVB-NEST interface

In [ ]:
tvb_spikeNet_model_builder = None

if np.all(SIM_MODE.lower() == "tvb-nest"):
    
    # Build a TVB-NEST interface with all the appropriate connections between the
    # TVB and NEST modelled regions
    
#     # ---------------------------- Opinionated TVB<->NEST interface builder----------------------------
#     from tvb_multiscale.tvb_nest.interfaces.models.jansen_rit import JansenRitTVBNESTInterfaceBuilder   
#     tvb_spikeNet_model_builder =  JansenRitTVBNESTInterfaceBuilder()  # opinionated builder
    
    # ---------------------------- Non opinionated TVB<->NEST interface builder----------------------------
    from tvb_multiscale.tvb_nest.interfaces.builders import TVBNESTInterfaceBuilder   
    tvb_spikeNet_model_builder =  TVBNESTInterfaceBuilder()  # non opinionated builder
    
    

In [ ]:
if tvb_spikeNet_model_builder is not None:

    tvb_spikeNet_model_builder.config = config
    tvb_spikeNet_model_builder.tvb_cosimulator = simulator            
    tvb_spikeNet_model_builder.spiking_network = nest_network
    # This can be used to set default tranformer and proxy models:
    tvb_spikeNet_model_builder.model = "RATE"          # "RATE" (or "SPIKES", "CURRENT") TVB->NEST interface
    tvb_spikeNet_model_builder.input_flag = True   # If True, NEST->TVB update will be implemented
    tvb_spikeNet_model_builder.output_flag = True  # If True, TVB->NEST coupling will be implemented
    # If default_coupling_mode = "TVB", large scale coupling towards spiking regions is computed in TVB
    # and then applied with no time delay via a single "TVB proxy node" / NEST device for each spiking region,
    # "1-to-1" TVB->NEST coupling.
    # If any other value, we need 1 "TVB proxy node" / NEST device for each TVB sender region node, and
    # large-scale coupling for spiking regions is computed in NEST, 
    # taking into consideration the TVB connectome weights and delays, 
    # in this "1-to-many" TVB->NEST coupling.
    tvb_spikeNet_model_builder.default_coupling_mode = TVB_NEST_COUPLING_MODE  # "NEST" # "TVB" 
    # Number of neurons per population to be used to compute population mean instantaneous firing rates:
    tvb_spikeNet_model_builder.N_E = nest_network.brain_regions[nest_nodes_inds[0]]["E"].number_of_neurons
    tvb_spikeNet_model_builder.N_Ein = nest_network.brain_regions[nest_nodes_inds[0]]["Ein"].number_of_neurons
    tvb_spikeNet_model_builder.N_Iin = nest_network.brain_regions[nest_nodes_inds[0]]["Iin"].number_of_neurons
    tvb_spikeNet_model_builder.proxy_inds = nest_nodes_inds
    # Set exclusive_nodes = True (Default) if the spiking regions substitute for the TVB ones:
    tvb_spikeNet_model_builder.exclusive_nodes = True  

    tvb_spikeNet_model_builder.output_interfaces = []
    tvb_spikeNet_model_builder.input_interfaces = []
    
   
    # options for a nonopinionated builder:
    from tvb_multiscale.core.interfaces.base.transformers.models.models import Transformers
    from tvb_multiscale.core.interfaces.base.transformers.builders import \
        DefaultTVBtoSpikeNetModels, DefaultSpikeNetToTVBModels, DefaultTVBtoSpikeNetTransformers
    from tvb_multiscale.tvb_nest.interfaces.builders import \
        TVBtoNESTModels, NESTInputProxyModels, DefaultTVBtoNESTModels, \
        NESTtoTVBModels, NESTOutputProxyModels, DefaultNESTtoTVBModels

    
    from tvb_multiscale.core.interfaces.base.transformers.models.jansen_rit import \
        DefaultTVBtoSpikeNetTransformersJansenRitCoupling, DefaultSpikeNetToTVBTransformersJansenRitInverseSigmoidal 
    
    def print_enum(enum):
        print("\n", enum)
        for name, member in enum.__members__.items():
            print(name,"= ", member.value)
    
    
    print("Available input (NEST->TVB update) / output (TVB->NEST coupling) interface models:")
    print_enum(TVBtoNESTModels)
    print_enum(NESTtoTVBModels)
    
    
    print("\n\nAvailable input (spikeNet->TVB update) / output (TVB->spikeNet coupling) transformer models:")

    print_enum(DefaultTVBtoSpikeNetModels)

    print("\n\nIn case coupling_mode = 'TVB':")
    print_enum(DefaultTVBtoSpikeNetTransformers)
    
    print("\n\nIn case coupling_mode = 'spikeNet or NEST':")
    print_enum(DefaultTVBtoSpikeNetTransformersJansenRitCoupling)
    
    print_enum(DefaultSpikeNetToTVBModels)
    print_enum(DefaultSpikeNetToTVBTransformersJansenRitInverseSigmoidal)    
    
    
    print("\n\nAvailable input (NEST->TVB update) / output (TVB->NEST coupling) proxy models:")

    print_enum(DefaultTVBtoNESTModels)
    print_enum(NESTInputProxyModels)
    
    print_enum(NESTOutputProxyModels)
    print_enum(DefaultNESTtoTVBModels)
    
    print("\n\nAll basic transformer models:")
    print_enum(Transformers)
    

In [ ]:
from copy import deepcopy

if tvb_spikeNet_model_builder is not None:

    # Using all default parameters for this example of an opinionated builder
    # tvb_spikeNet_model_builder.default_config()
   
    
    # or setting a nonopinionated builder:
    from tvb_multiscale.core.interfaces.tvb.interfaces import TVBtoSpikeNetModels
    tvb_spikeNet_model_builder._spikeNet_to_tvb_transformer_models = DefaultSpikeNetToTVBTransformersJansenRitInverseSigmoidal
    if tvb_spikeNet_model_builder.default_coupling_mode != "TVB":
        # This is the case where TVB -> NEST coupling is summed up in NEST, 
        # via 1 proxy per sending TVB node.
        # Therefore, we need a dedicated JansenRit Sigmoidal Coupling transformer
        # to compute the corresponding sigmoidal coupling of the y1-y2 quantity.
        tvb_spikeNet_model_builder._tvb_to_spikeNet_transformer_models = DefaultTVBtoSpikeNetTransformersJansenRitCoupling
        voi = np.array(["y0", "y1", "y2"])
        proxy_inds = np.arange(simulator.connectivity.number_of_regions).astype('i')
        proxy_inds = np.delete(proxy_inds, nest_nodes_inds)
    else:
        # This is the case where TVB -> NEST coupling is computed as TVB large scale coupling
        # and then sent to NEST with 1 TVB proxy node per receiving NEST spiking node.
        # quantity transfered to NEST can be linearly transformed to a rate quantity.
        voi = np.array(["y0"])
        proxy_inds = nest_nodes_inds
    # Interface where sigmoidal coupling is already computed in TVB:
    tvb_spikeNet_model_builder.output_interfaces = \
        [{'voi': voi,        # TVB state variable to get data from
          'populations': np.array(["E"]), # NEST populations to couple to
        # --------------- Arguments that can default if not given by the user:------------------------------
          'model': 'RATE',                # This can be used to set default tranformer and proxy models
          'coupling_mode': tvb_spikeNet_model_builder.default_coupling_mode,         # or "TVB", spikeNet", "NEST", etc
          'proxy_inds': proxy_inds,  # TVB proxy region nodes' indices
          # Set the enum entry or the corresponding label name for the "proxy_model", 
          # or import and set the appropriate NEST proxy device class, e.g., NESTInhomogeneousPoissonGeneratorSet, directly
          # options: "RATE", "RATE_TO_SPIKES", SPIKES", "PARROT_SPIKES" or CURRENT"
          # see tvb_multiscale.tvb_nest.interfaces.io.NESTInputProxyModels for options and related NESTDevice classes, 
          # and tvb_multiscale.tvb_nest.interfaces.io.DefaultTVBtoNESTModels for the default choices
          'proxy_model': "RATE",  
          # Set the enum entry or the corresponding label name for the "transformer_model", 
          # or import and set the appropriate tranformer class, e.g., ScaleRate, directly
          # options: "RATE", "SPIKES", "SPIKES_SINGE_INTERACTION", "SPIKES_MULTIPLE_INTERACTION", "CURRENT"
          # see tvb_multiscale.core.interfaces.base.transformers.models.DefaultTVBtoSpikeNetTransformers for options and related Transformer classes,
          # and tvb_multiscale.core.interfaces.base.transformers.models.DefaultTVBtoSpikeNetModels for default choices
          'transformer_model': "RATE", # DefaultTVBtoSpikeNetTransformers.JANSEN_RIT_COUPLING_RATE,  
          'spiking_proxy_inds': nest_nodes_inds  # Same as "proxy_inds" for this kind of interface
         }
        ]
    
    for interface in tvb_spikeNet_model_builder.output_interfaces:
        # The "scale_factor" scales the TVB state variable to convert it to an 
        # instantaneous rate:
        if tvb_spikeNet_model_builder.model == TVBtoSpikeNetModels.SPIKES.name:
            # The "number_of_neurons" will determine how many spike trains will be generated:
            interface["transformer_params"] = \
                    {"scale_factor": np.array([100]),
                     "number_of_neurons": np.array([tvb_spikeNet_model_builder.N_Ein])}
        else:  # RATE
            # Here the rate is a total rate, assuming a number of sending neurons:
            interface["transformer_params"] = {"scale_factor": 100 * np.array([tvb_spikeNet_model_builder.N_Ein])}
                
    for pop, sv in zip(["E", "Ein", "Iin"], ["y0", "y1", "y2"]):
        tvb_spikeNet_model_builder.input_interfaces.append(
           {'voi': np.array([sv]),
            'populations': np.array([pop]),
            'proxy_inds': nest_nodes_inds,
            # --------------- Arguments that can default if not given by the user:------------------------------
            # Set the enum entry or the corresponding label name for the "proxy_model", 
            # or import and set the appropriate NEST proxy device class, e.g., NESTSpikeRecorderMeanSet, directly
            # options "SPIKES" (i.e., spikes per neuron), "SPIKES_MEAN", "SPIKES_TOTAL" 
            # (the last two are identical for the momentm returning all populations spikes together)
            # see tvb_multiscale.tvb_nest.interfaces.io.NESTOutputProxyModels for options and related NESTDevice classes, 
            # and tvb_multiscale.tvb_nest.interfaces.io.DefaultNESTtoTVBModels for the default choices
            'proxy_model': "%s_MEAN" % NEST_TVB_MODEL, # "POTENTIAL_MEAN",  # "SPIKES_MEAN",  
            # Set the enum entry or the corresponding label name for the "transformer_model", 
            # or import and set the appropriate tranformer class, e.g., ElephantSpikesHistogramRate, directly
            # options: "SPIKES", "SPIKES_TO_RATE", "SPIKES_TO_HIST", "SPIKES_TO_HIST_RATE"
            # see tvb_multiscale.core.interfaces.base.transformers.models.DefaultSpikeNetToTVBTransformers for options and related Transformer classes,
            # and tvb_multiscale.core.interfaces.base.transformers.models.DefaultSpikeNetToTVBModels for default choices
            # Options for 'transformer_model': NEST_TVB_MODEL #'POTENTIAL' # 'SPIKES'
            'transformer_model': NEST_TVB_MODEL  # DefaultSpikeNetToTVBTransformersJansenRitInverseSigmoidal.SPIKES_LINEAR_INTEGRATION  
           }
        )
    
    if NEST_TVB_MODEL == "SPIKES":
        Rmax = 16.0 # Hz
        Rmin = 0.1 # Hz
        ampl0 = np.log(2*Rmax/Rmin - 1)
        for interface, N, maxim, ampl in zip(tvb_spikeNet_model_builder.input_interfaces, 
                                        [tvb_spikeNet_model_builder.N_E, 
                                         tvb_spikeNet_model_builder.N_Ein, 
                                         tvb_spikeNet_model_builder.N_Iin], 
                                        [1.5, 25.0, 70.0], 
                                        [3.0, 10.0, 70.0]):

                # The "scale_factor" scales the instantaneous rate coming from NEST, before setting it to TVB,
                # in our case converting the rate to a mean reate 
                # and scaling it to be in the TVB model's state variable range
                # Spike count to rate linear transform: 
                # rate_i = scale_factor_i * spike_count_i + translation_factor_i
                # Scale factor such as to transform units from spikes per dt ms -> spikes per sec 
                # and then divide by the number of neurons for a mean population rate
                # Inverse sigmoidal rate to potential transform:
                # y_i = v0_i - log(2Rmax_i/(maximum(minimum(rate_i, Rmin_i), Rmax_i)) - 1)/r_i
                interface["transformer_params"] = {"scale_factor": np.array([1e-3])/simulator.integrator.dt/N,
                                                   "Rmax": np.array([Rmax]), 
                                                   "Rmin": np.array([Rmin]),
                                                   "v0": np.array([maxim]),
                                                   "r": np.array([ampl0 / ampl])}
                if interface['transformer_model'] == \
                    DefaultSpikeNetToTVBTransformersJansenRitInverseSigmoidal.SPIKES_LINEAR_INTEGRATION:
                    interface["transformer_params"]["tau"] = np.array([1.0])
                    
    else:
        for interface, pop, ampl, baseline in \
            zip(tvb_spikeNet_model_builder.input_interfaces, 
                nest_network.brain_regions[0],
               [3.0, 20.0, 70.0], 
               [0.0, 15.0, 35.0]):
            vmin = np.mean(pop.Get("E_L")['E_L'])
            vmax = np.mean(pop.Get("V_th")['V_th'])
            vrange = vmax - vmin
            vbase = vmin + vrange/2
            scale = ampl/vrange
            translation = baseline - scale*vbase
            interface["transformer_params"] = {"scale_factor": np.array([scale]),
                                               "translation_factor": np.array([translation])} 
            # interface['proxy_params'] = {"record_to": "ascii"}
            
        
    
    # Configure and build:
    tvb_spikeNet_model_builder.configure()
    # tvb_spikeNet_model_builder.print_summary_info_details(recursive=1)
    
    print("output (TVB->NEST coupling) interfaces' configurations:\n")
    display(tvb_spikeNet_model_builder.output_interfaces)
    
    print("\ninput (NEST->TVB update) interfaces' configurations:\n")
    display(tvb_spikeNet_model_builder.input_interfaces)
    
    
        

In [ ]:
if tvb_spikeNet_model_builder is not None:
    simulator = tvb_spikeNet_model_builder.build()

    simulator.simulate_spiking_simulator = nest_network.nest_instance.Run  # set the method to run NEST

    # simulator.print_summary_info(recursive=3)
    # simulator.print_summary_info_details(recursive=3)

    print("\n\noutput (TVB->NEST coupling) interfaces:\n")
    simulator.output_interfaces.print_summary_info_details(recursive=2)
    
    print("\n\ninput (NEST->TVB update) interfaces:\n")
    simulator.input_interfaces.print_summary_info_details(recursive=2)
    

## 4. Configure simulator, simulate, gather results

In [ ]:
# Configure the simulator with the TVB-NEST interface...
# ...and simulate!

# Set the simulation time:
simulator.simulation_length = 1100.0
tic = time.time()
if np.all(SIM_MODE.lower() == "tvb"):
    # For TVB 
    results = simulator.run()
else:
    nest_network.nest_instance.Prepare()
    nest_simulation_length = simulator.simulation_length
    if NEST_TVB_MODEL == "POTENTIAL":
        # Run an initial spiking step so that multimeters return outputs for the correct times:
        nest_network.nest_instance.Run(nest_network.nest_instance.GetKernelStatus("resolution"))
    if np.all(SIM_MODE.lower() == "nest"):
        print("Simulating only NEST...")
        # Integrate NEST for simulation_length + 1 NEST time step so that multimeters get the last time point
        # unless you plan to continue simulation later
        nest_network.nest_instance.Run(simulator.simulation_length)
        results = None
    else:
        print("Simulating TVB-NEST...")
        simulator.configure()
        # Adjust simulation length to be an integer multiple of synchronization_time:
        simulator.simulation_length = \
                np.ceil(simulator.simulation_length / simulator.synchronization_time) * simulator.synchronization_time
        results = simulator.run()
        #  Cleanup NEST network unless you plan to continue simulation later
        nest_network.nest_instance.Cleanup()
    
print("\nSimulated in %f secs!" % (time.time() - tic))


## 5. Plot results and write them to HDF5 files

In [ ]:
from scipy.io import savemat
# set to False for faster plotting of only mean field variables and dates, apart from spikes" rasters:
plot_per_neuron = False  
MAX_VARS_IN_COLS = 3
MAX_REGIONS_IN_ROWS = 10
MIN_REGIONS_FOR_RASTER_PLOT = 9

# Set the transient time to be optionally removed from results:
simulation_length = simulator.simulation_length
transient = 0.1 * simulation_length 

# from examples.plot_write_results import plot_write_results
# plot_write_results(results, simulator, populations=populations, 
#                    transient=transient, tvb_state_variable_type_label="State Variables", 
#                    tvb_state_variables_labels=simulator.model.variables_of_interest, 
#                    plot_per_neuron=plot_per_neuron, plotter=plotter, config=config)

### TVB plots

In [ ]:
# If you want to see what the function above does, take the steps, one by one
try:
    # We need framework_tvb for writing and reading from HDF5 files
    from tvb_multiscale.core.tvb.io.h5_writer import H5Writer
    from examples.plot_write_results import write_RegionTimeSeriesXarray_to_h5
    writer = H5Writer()
    
except:
    writer = None
    
# Put the results in a Timeseries instance
from tvb.contrib.scripts.datatypes.time_series_xarray import TimeSeriesRegion as TimeSeriesXarray

source_ts = None
t = simulation_length * simulator.integrator.dt
if results is not None:
    # Substitute with TimeSeriesRegion fot TVB like functionality:
    # from tvb.contrib.scripts.datatypes.time_series import TimeSeriesRegion
    source_ts = TimeSeriesXarray(  
            data=results[0][1], time=results[0][0],
            connectivity=simulator.connectivity,
            labels_ordering=["Time", "State Variable", "Region", "Neurons"],
            labels_dimensions={"State Variable": list(simulator.model.variables_of_interest),
                               "Region": simulator.connectivity.region_labels.tolist()},
            sample_period=simulator.integrator.dt)
    source_ts.configure()

    t = source_ts.time

    # Write to file
    if writer:
        write_RegionTimeSeriesXarray_to_h5(source_ts, writer,
                                           os.path.join(config.out.FOLDER_RES, source_ts.title)+".h5")
    source_ts   
    

In [ ]:
# Plot TVB time series
if source_ts is not None:
    source_ts.plot_timeseries(plotter_config=plotter.config, 
                          hue="Region" if source_ts.shape[2] > MAX_REGIONS_IN_ROWS else None, 
                          per_variable=source_ts.shape[1] > MAX_VARS_IN_COLS, 
                          figsize=FIGSIZE);

In [ ]:
# Focus on the nodes modelled in NEST: raster plot
if source_ts is not None and source_ts.number_of_labels > MIN_REGIONS_FOR_RASTER_PLOT:
    source_ts.plot_raster(plotter_config=plotter.config, 
                          per_variable=source_ts.shape[1] > MAX_VARS_IN_COLS,
                          figsize=FIGSIZE, figname="Spiking nodes TVB Time Series Raster");

In [ ]:
# Focus on the nodes modelled in NEST: 
n_spiking_nodes = len(nest_nodes_inds)
if source_ts is not None and n_spiking_nodes:
    source_ts_nest = source_ts[:, :, nest_nodes_inds]
    source_ts_nest.plot_timeseries(plotter_config=plotter.config, 
                                   hue="Region" if source_ts_nest.shape[2] > MAX_REGIONS_IN_ROWS else None, 
                                   per_variable=source_ts_nest.shape[1] > MAX_VARS_IN_COLS, 
                                   figsize=FIGSIZE, figname="Spiking nodes TVB Time Series");

In [ ]:
# Focus on the nodes modelled in NEST: raster plot
if source_ts is not None and n_spiking_nodes: # and source_ts_nest.number_of_labels > MIN_REGIONS_FOR_RASTER_PLOT:
    source_ts_nest.plot_raster(plotter_config=plotter.config, 
                               per_variable=source_ts_nest.shape[1] > MAX_VARS_IN_COLS,
                               figsize=FIGSIZE, figname="Spiking nodes TVB Time Series Raster");

### Interactive time series plot

In [ ]:
# # ...interactively as well
# # For interactive plotting:
# %matplotlib notebook 
# plotter.plot_timeseries_interactive(source_ts)

### Spiking Network plots

In [ ]:
spikeNet_analyzer = None
if nest_network is not None:
    from tvb_multiscale.core.data_analysis.spiking_network_analyser import SpikingNetworkAnalyser
    # Create a SpikingNetworkAnalyzer:
    spikeNet_analyzer = \
        SpikingNetworkAnalyser(spikeNet=nest_network,
                               start_time=0.0, end_time=simulation_length, 
                               transient=transient, time_series_output_type="TVB", 
                               return_data=True, force_homogeneous_results=True, 
                               period=simulator.monitors[0].period, connectivity=simulator.connectivity
                              )


### Plot spikes' raster and mean spike rates and correlations

In [ ]:
spikes_res = None
if spikeNet_analyzer is not None:
    # Spikes rates and correlations per Population and Region
    spikes_res = \
        spikeNet_analyzer.\
            compute_spikeNet_spikes_rates_and_correlations(
                populations_devices=None, regions=None,
                rates_methods=[], rates_kwargs=[{}],rate_results_names=[],
                corrs_methods=[], corrs_kwargs=[{}], corrs_results_names=[], bin_kwargs={},
                data_method=spikeNet_analyzer.get_spikes_from_device, data_kwargs={},
                return_devices=False
            );


In [ ]:
if spikes_res:
    print(spikes_res["mean_rate"])
    print(spikes_res["spikes_correlation_coefficient"])
    # Plot spikes' rasters together with mean population's spikes' rates' time series
    if plotter:
        plotter.plot_spike_events(spikes_res["spikes"], mean_results=spikes_res["mean_rate"], # time_series=spikes_res["mean_rate_time_series"], 
                                  figsize=(20, 22),  
                                  stimulus=None,
                                  stimulus_linewidth=5.0,
                                  spikes_markersize=0.5, spikes_alpha=0.5,
                                  n_y_ticks=3, n_time_ticks=5, show_time_axis=True, 
                                  time_axis_min=0.0, time_axis_max=simulation_length)
        from tvb_multiscale.core.plot.correlations_plot import plot_correlations
        plot_correlations(spikes_res["spikes_correlation_coefficient"], plotter)

In [ ]:
if spikes_res:
    print("Mean spike rates:")
    for pop in spikes_res["mean_rate"].coords["Population"]:
        for reg in spikes_res["mean_rate"].coords["Region"]:
            if not np.isnan(spikes_res["mean_rate"].loc[pop, reg]):
                print("%s - %s: %g" % (pop.values.item().split("_spikes")[0], reg.values.item(), 
                                       spikes_res["mean_rate"].loc[pop, reg].values.item()))

    # savemat(os.path.join(config.out.FOLDER_RES, "spikes_mean_rates.mat"), spikes_res["mean_rate"].to_dict())


# Mean spike rates:
# E - bankssts_L: 15.8543
# E - bankssts_R: 15.7639
# Ein - bankssts_L: 13.0695
# Ein - bankssts_R: 13.0695
# Iin - bankssts_L: 13.0695
# Iin - bankssts_R: 13.0695

In [ ]:
spikes_sync = None

if spikeNet_analyzer is not None:

    spikeNet_analyzer.resample = True
    spikes_sync = \
        spikeNet_analyzer.compute_spikeNet_synchronization(populations_devices=None, regions=None,
                                                           comp_methods=[spikeNet_analyzer.compute_spikes_sync, 
                                                                         spikeNet_analyzer.compute_spikes_sync_time_series, 
                                                                         spikeNet_analyzer.compute_spikes_distance, 
                                                                         spikeNet_analyzer.compute_spikes_distance_time_series,
                                                                         spikeNet_analyzer.compute_spikes_isi_distance, 
                                                                         spikeNet_analyzer.compute_spikes_isi_distance_time_series],
                                                           computations_kwargs=[{}], data_kwargs={},
                                                           return_spikes_trains=False, return_devices=False)
# print(spikes_sync)


In [ ]:
if spikes_sync:
    plotter.config.FONTSIZE = 20 # plotter.config.LARGE_FONTSIZE  # LARGE = 12, default = 10
    plotter.plot_spike_events(spikes_res["spikes"], 
                              time_series=spikes_sync["spikes_sync_time_series"], 
                              mean_results=spikes_sync["spikes_sync"], 
                              stimulus_linewidth=5.0,
                              spikes_markersize=0.5, spikes_alpha=0.5,
                              n_y_ticks=3, n_time_ticks=5, show_time_axis=True, 
                              time_axis_min=0.0, time_axis_max=simulation_length, figsize=(20, 22)
                              )

In [ ]:
if spikes_sync:
    plotter.config.FONTSIZE = 20 # plotter.config.LARGE_FONTSIZE  # LARGE = 12, default = 10
    plotter.plot_spike_events(spikes_res["spikes"], 
                              time_series=spikes_sync["spikes_distance_time_series"], 
                              mean_results=spikes_sync["spikes_distance"], 
                              stimulus_linewidth=5.0,
                              spikes_markersize=0.5, spikes_alpha=0.5,
                              n_y_ticks=3, n_time_ticks=5, show_time_axis=True, 
                              time_axis_min=0.0, time_axis_max=simulation_length, figsize=(20, 22)
                                     )

In [ ]:
if spikes_sync:
    plotter.config.FONTSIZE = 20 # plotter.config.LARGE_FONTSIZE  # LARGE = 12, default = 10
    plotter.plot_spike_events(spikes_res["spikes"], 
                              time_series=spikes_sync["spikes_isi_distance_time_series"], 
                              mean_results=spikes_sync["spikes_isi_distance"], 
                              stimulus_linewidth=5.0,
                              spikes_markersize=0.5, spikes_alpha=0.5,
                              n_y_ticks=3, n_time_ticks=5, show_time_axis=True, 
                              time_axis_min=0.0, time_axis_max=simulation_length, figsize=(20, 22)
                                     )

In [ ]:
if spikes_sync:
    print("Spike synchronization:")
    for pop in spikes_sync["spikes_sync"].coords["Population"]:
        for reg in spikes_sync["spikes_sync"].coords["Region"]:
            if not np.isnan(spikes_sync["spikes_sync"].loc[pop, reg]):
                print("%s - %s: %g" % (pop.values.item().split("_spikes")[0], reg.values.item(), 
                                       spikes_sync["spikes_sync"].loc[pop, reg].values.item()))

#     savemat(os.path.join(config.out.FOLDER_RES, "spikes_sync.mat"), spikes_sync["spikes_sync"].to_dict())
#     savemat(os.path.join(config.out.FOLDER_RES, "spikes_sync_time_series.mat"), spikes_sync["spikes_sync_time_series"].to_dict())

In [ ]:
if spikes_sync:
    print("Spike distance:")
    for pop in spikes_sync["spikes_distance"].coords["Population"]:
        for reg in spikes_sync["spikes_distance"].coords["Region"]:
            if not np.isnan(spikes_sync["spikes_distance"].loc[pop, reg]):
                print("%s - %s: %g" % (pop.values.item().split("_spikes")[0], reg.values.item(), 
                                       spikes_sync["spikes_distance"].loc[pop, reg].values.item()))

#     savemat(os.path.join(config.out.FOLDER_RES, "spikes_distance.mat"), spikes_sync["spikes_distance"].to_dict())
#     savemat(os.path.join(config.out.FOLDER_RES, "spikes_distance_time_series.mat"), spikes_sync["spikes_distance_time_series"].to_dict())

In [ ]:
if spikes_sync:
    print("Spike ISI distance:")
    for pop in spikes_sync["spikes_isi_distance"].coords["Population"]:
        for reg in spikes_sync["spikes_isi_distance"].coords["Region"]:
            if not np.isnan(spikes_sync["spikes_isi_distance"].loc[pop, reg]):
                print("%s - %s: %g" % (pop.values.item().split("_spikes")[0], reg.values.item(), 
                                       spikes_sync["spikes_isi_distance"].loc[pop, reg].values.item()))

#     savemat(os.path.join(config.out.FOLDER_RES, "spikes_isi_distance.mat"), spikes_sync["spikes_isi_distance"].to_dict())
#     savemat(os.path.join(config.out.FOLDER_RES, "spikes_isi_distance_time_series.mat"), spikes_sync["spikes_isi_distance_time_series"].to_dict())

In [ ]:
if spikes_res and writer:
    writer.write_object(spikes_res["spikes"].to_dict(), 
                        path=os.path.join(config.out.FOLDER_RES,  "Spikes") + ".h5");
    writer.write_object(spikes_res["mean_rate"].to_dict(),
                        path=os.path.join(config.out.FOLDER_RES,
                                          spikes_res["mean_rate"].name) + ".h5");
    write_RegionTimeSeriesXarray_to_h5(spikes_res["mean_rate_time_series"], writer,
                                       os.path.join(config.out.FOLDER_RES,
                                                    spikes_res["mean_rate_time_series"].title + ".h5"),
                                       recursive=False);
    writer.write_object(spikes_res["spikes_correlation_coefficient"].to_dict(),
                        path=os.path.join(config.out.FOLDER_RES,
                                          spikes_res["spikes_correlation_coefficient"].name) + ".h5");

### Get  SpikingNetwork mean field variable time series and plot them

In [ ]:
# Continuous time variables' data of spiking neurons
spikeNet_ts = None
mean_field_ts = None
if spikeNet_analyzer:
    if plot_per_neuron:
        spikeNet_analyzer.return_data = True
    else:
        spikeNet_analyzer.return_data = False
    spikeNet_ts = \
        spikeNet_analyzer. \
             compute_spikeNet_mean_field_time_series(populations_devices=None, regions=None, variables=None,
                                                     computations_kwargs={}, data_kwargs={}, return_devices=False)
    if spikeNet_ts:
        if plot_per_neuron:
            mean_field_ts = spikeNet_ts["mean_field_time_series"]  # mean field
            spikeNet_ts = spikeNet_ts["data_by_neuron"]  # per neuron data
        else:
            mean_field_ts = spikeNet_ts
            spikeNet_ts = None
        if mean_field_ts and mean_field_ts.size > 0:
            mean_field_ts.plot_timeseries(plotter_config=plotter.config, 
                                          per_variable=mean_field_ts.shape[1] > MAX_VARS_IN_COLS)
            if mean_field_ts.number_of_labels > MIN_REGIONS_FOR_RASTER_PLOT:
                mean_field_ts.plot_raster(plotter_config=plotter.config, 
                                          per_variable=mean_field_ts.shape[1] > MAX_VARS_IN_COLS,
                                          linestyle="--", alpha=0.5, linewidth=0.5)


In [ ]:
# Write results to file:
if mean_field_ts and writer:
    write_RegionTimeSeriesXarray_to_h5(mean_field_ts, writer,
                                       os.path.join(config.out.FOLDER_RES, mean_field_ts.title + ".h5"), 
                                       recursive=False)

### Compute per neuron spikes' rates times series and plot them

In [ ]:
if spikes_res and plot_per_neuron:
    from tvb.simulator.plot.base_plotter import pyplot
    spikeNet_analyzer.return_data = False
    rates_ts_per_neuron = \
        spikeNet_analyzer. \
            compute_spikeNet_rates_time_series(populations_devices=None, regions=None,
                                               computations_kwargs={}, data_kwargs={},
                                               return_spikes_trains=False, return_devices=False);
    if rates_ts_per_neuron is not None and rates_ts_per_neuron.size:
        # Regions in rows
        row = rates_ts_per_neuron.dims[2] if rates_ts_per_neuron.shape[2] > 1 else None
        if row is None:
            # Populations in rows
            row = rates_ts_per_neuron.dims[1] if rates_ts_per_neuron.shape[1] > 1 else None
            col = None
        else:
            # Populations in columns
            col = rates_ts_per_neuron.dims[1] if rates_ts_per_neuron.shape[1] > 1 else None
        pyplot.figure()
        rates_ts_per_neuron.plot(y=rates_ts_per_neuron.dims[3], row=row, col=col, cmap="jet")
        plotter.base._save_figure(figure_name="Spike rates per neuron")
        # del rates_ts_per_neuron # to free memory

### Plot per neuron SpikingNetwork time series

In [ ]:
# Regions in rows
if spikeNet_ts is not None and spikeNet_ts.size:
    row = spikeNet_ts.dims[2] if spikeNet_ts.shape[2] > 1 else None
    if row is None:
        # Populations in rows
        row = spikeNet_ts.dims[3] if spikeNet_ts.shape[3] > 1 else None
        col = None
    else:
        # Populations in cols
         col = spikeNet_ts.dims[3] if spikeNet_ts.shape[3] > 1 else None
    for var in spikeNet_ts.coords[spikeNet_ts.dims[1]]:
        this_var_ts = spikeNet_ts.loc[:, var, :, :, :]
        this_var_ts.name = var.item()
        pyplot.figure()
        this_var_ts.plot(y=spikeNet_ts.dims[4], row=row, col=col, cmap="jet", figsize=FIGSIZE)
        plotter.base._save_figure(
            figure_name="Spiking Network variables' time series per neuron: %s" % this_var_ts.name)
    del spikeNet_ts # to free memory

# References

1 Paula Sanz Leon, Stuart A. Knock, M. Marmaduke Woodman, Lia Domide, <br>
  Jochen Mersmann, Anthony R. McIntosh, Viktor Jirsa (2013) <br>
  The Virtual Brain: a simulator of primate brain network dynamics. <br>
  Frontiers in Neuroinformatics (7:10. doi: 10.3389/fninf.2013.00010) <br>
  https://www.thevirtualbrain.org/tvb/zwei <br>
  https://github.com/the-virtual-brain <br>

2 Ritter P, Schirner M, McIntosh AR, Jirsa VK. 2013.  <br>
  The Virtual Brain integrates computational modeling  <br>
  and multimodal neuroimaging. Brain Connectivity 3:121–145. <br>

3 Jordan, Jakob; Mørk, Håkon; Vennemo, Stine Brekke;   Terhorst, Dennis; Peyser, <br>
  Alexander; Ippen, Tammo; Deepu, Rajalekshmi;   Eppler, Jochen Martin; <br>
  van Meegen, Alexander;   Kunkel, Susanne; Sinha, Ankur; Fardet, Tanguy; Diaz, <br>
  Sandra; Morrison, Abigail; Schenck, Wolfram; Dahmen, David;   Pronold, Jari; <br>
  Stapmanns, Jonas;   Trensch, Guido; Spreizer, Sebastian;   Mitchell, Jessica; <br>
  Graber, Steffen; Senk, Johanna; Linssen, Charl; Hahne, Jan; Serenko, Alexey; <br>
  Naoumenko, Daniel; Thomson, Eric;   Kitayama, Itaru; Berns, Sebastian;   <br>
  Plesser, Hans Ekkehard <br>
  NEST is a simulator for spiking neural network models that focuses <br>
  on the dynamics, size and structure of neural systems rather than on <br>
  the exact morphology of individual neurons. <br>
  For further information, visit http://www.nest-simulator.org. <br>
  The release notes for this release are available at  <br>
  https://github.com/nest/nest-simulator/releases/tag/v2.18.0 <br>